### **Backend**

setup

In [158]:
#!pip install selenium
#!pip install SpeechRecognition
#!pip install pyaudio

In [2]:
import random
import time

from selenium import webdriver
from selenium.webdriver.common.by import By

import speech_recognition as sr
import openai

class Avatar:
    def __init__(self, name='{self.name}', notebook_token ='') -> None:
        self.name = name

        driver=webdriver.Chrome('/Users/jsedoc/Downloads/chromedriver')
        driver.get('http://localhost:3000/')

        self.textarea = driver.find_element(By.TAG_NAME, "textarea")
        self.button = driver.find_element(By.TAG_NAME, "button")

        self.r = sr.Recognizer()
        with sr.Microphone() as source:                # use the default microphone as the audio source
            self.r.adjust_for_ambient_noise(source)         # listen for 1 second to calibrate the energy threshold for ambient noise levels


        self.wps = 2.2 # words per second

        self.conversation = []

        self.nbc = NotebookController(notebook_token)
        
    def speak(self, text, all_text=''):
        self.textarea.clear()
        self.textarea.send_keys(text)
        self.button.click()
        if all_text == '':
            self.conversation.append(text)
        else:
            self.conversation.append(all_text)
        # 1 sec start time and the number of words in the text divided by wps (average words per second)
        time.sleep(1 + len(text.split())/self.wps)
        #print('done talking')

    def listen(self):
        with sr.Microphone() as source:
            audio = self.r.listen(source)
            try:
                text = self.r.recognize_google(audio)
                self.conversation.append(text)
                spoken_text, all_text = self.generate_response(text)
                self.speak(spoken_text, all_text)
            except KeyError:
                print("Sorry, I didn't get that")
                if random.randint(1, 5) == 1:
                    self.speak("hmmm ... I'm so sorry, I didn't get that")

    def text_input(self, text):
        self.conversation.append(text)
        spoken_text, all_text = self.generate_response(text)
        self.speak(spoken_text, all_text)

    def format_for_GPT(self):
        output_string = ''
        for i, message in enumerate(self.conversation):
            if i % 2 == 0:
                # Even index, message is from the user                                                                                  
                output_string += f'You: {message}\n'
            else:
                # Odd index, message is from {self.name}                                                                                        
                output_string += f'{self.name}: {message}\n'
        return output_string

    def extract_code(self, text):
        code = ''
        if '```' in text:
            code = text.split('```')[1]
        return code
    
    def remove_code(self, text):
        if '```' in text:
            text = text.split('```')[0]
        return text

    def generate_response(self, text):                                                                                                
        # Implement chatbot's response generation logic here                                                                       
                                                                                                                                        
        openai.api_key = 'API_KEY'
                                                                                                                                        
        conv_str = self.format_for_GPT()                                                                                       
                                                                                                                                                                                                                                    
        prompt=f"{self.name} is a chatbot. {self.name} responds like an expert data scientist. {self.name} has read Data Science for Business in preparation for the interview.\n\n{conv_str}\n{self.name}: "
        prompt=f"{self.name} is a chatbot. {self.name} responds like an data scientist in a coding exam. {self.name} pust ``` before and after Python code.\n\n{conv_str}\n{self.name}: "
                                                                                                                                        
        #print(prompt)                                                                                                                  
                                                                                                                                        
        response = openai.Completion.create(                                                                                            
            model="text-davinci-003",                                                                                                   
            prompt=prompt,                                                                                                              
            temperature=0.5,                                                                                                            
            max_tokens=289,                                                                                                             
            top_p=0.3,                                                                                                                  
            frequency_penalty=0.5,                                                                                                      
            presence_penalty=0.3,                                                                                                       
            stop=["You:"]                                                                                                               
        )                                                                                                                               
                                                                                                                                        
        response_text = response.choices[0].text.replace('{self.name}: ', '')

        code = self.extract_code(response_text)
        if code:
            self.nbc.run_code(code)
            self.nbc.new_cell()
            spoken_text = self.remove_code(response_text)
        else:
            spoken_text = response_text
                                                      
        return spoken_text, response_text 
    

In [6]:
ad = Avatar(name='DS', notebook_token='de3bd468afafc97d2070c3ca5a2e88388c70c7d6e4be84bd')
time.sleep(10)
ad.speak("Hello, I am DS. It's a pleasure to meet you.")
#ad.listen()

/var/folders/b1/zt36t5217xl_7s5k38c836tc0000gn/T/ipykernel_12678/661580032.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('/Users/jsedoc/Downloads/chromedriver')


http://localhost:8889/notebooks/GPT3_notebook.ipynb?token=de3bd468afafc97d2070c3ca5a2e88388c70c7d6e4be84bd


/var/folders/b1/zt36t5217xl_7s5k38c836tc0000gn/T/ipykernel_12678/4280353644.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.notebook_driver=webdriver.Chrome('/Users/jsedoc/Downloads/chromedriver')


In [22]:
while 1:
    text = input()
    if text == "[END]":
        break
    ad.text_input(text)

In [7]:
ad.listen()

result2:
{   'alternative': [   {   'confidence': 0.97067946,
                           'transcript': 'hi I can you tell me a little bit '
                                         'more about data science are you '
                                         'ready for the interview'},
                       {   'transcript': 'hi I can you tell me a little bit '
                                         'more about data science or you ready '
                                         'for the interview'},
                       {   'transcript': 'hi I can you tell me a little bit '
                                         'more about data science so you ready '
                                         'for the interview'}],
    'final': True}


In [125]:
# To reset the conversation
#ad.conversation =[]

## Notebook code

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time

class NotebookController:
    def __init__(self, token) -> None:
        notebook_url = f"http://localhost:8889/notebooks/GPT3_notebook.ipynb?token={token}"
        print(notebook_url)
        self.notebook_driver=webdriver.Chrome('/Users/jsedoc/Downloads/chromedriver')
        self.notebook_driver.get(notebook_url)
    
    def run_code(self, code_text, cell_location=-1):
        cell = self.notebook_driver.find_elements(By.CLASS_NAME, "input_area")
        cell = cell[cell_location]
        cell.click()

        # Find the first code cell
        cell = self.notebook_driver.find_elements(By.TAG_NAME, "textarea")
        cell = cell[cell_location]

        # Enter some code into the cell
        #cell.clear()
        cell.send_keys(Keys.COMMAND + 'a')
        cell.send_keys(Keys.DELETE)
        cell.send_keys(code_text)

        cell.send_keys(Keys.CONTROL, Keys.ENTER)

    def run_all_cells(self):
        run_all_cells = self.notebook_driver.find_element(By.ID, "run_all_cells")
        run_all_cells.click()

    def get_output(self, cell_location=-1):
        cell = self.notebook_driver.find_elements(By.CLASS_NAME, "output_area")
        cell = cell[cell_location]
        return cell.text

    def new_cell(self):
        #new_cell = notebook_driver.find_element(By.ID, "insert_cell_below")
        #new_cell.click()
        self.notebook_driver.find_element(By.XPATH, "//button[@title='insert cell below']").click()
    
    def delete_cell(self):
        self.notebook_driver.find_element(By.XPATH, "//button[@title='cut selected cells']").click()

    def restart_kernel(self):
        self.notebook_driver.execute_script("Jupyter.notebook.restart_kernel()")
        time.sleep(1)
        self.notebook_driver.find_element(By.XPATH, "//button[@class='btn btn-default btn-sm btn-danger']").click()
        time.sleep(1)

    def close(self):
        self.notebook_driver.close()

        

In [62]:
nbc = NotebookController('de3bd468afafc97d2070c3ca5a2e88388c70c7d6e4be84bd')

http://localhost:8889/notebooks/GPT3_notebook.ipynb?token=de3bd468afafc97d2070c3ca5a2e88388c70c7d6e4be84bd


/var/folders/b1/zt36t5217xl_7s5k38c836tc0000gn/T/ipykernel_96518/697486256.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.notebook_driver=webdriver.Chrome('/Users/jsedoc/Downloads/chromedriver')


In [55]:
nbc.run_code('import pandas as pd')

In [56]:
nbc.new_cell()

In [57]:
nbc.delete_cell()

In [60]:
nbc.close()

In [63]:
nbc.restart_kernel()